In [1]:
# 0. Login no Hugging Face (Para baixar o modelo)
from huggingface_hub import login
# Se pedir input, cole seu token e dê Enter.
login()

# 1. Instalar bibliotecas necessárias
# (O bitsandbytes e accelerate são essenciais para rodar com pouca memória)
!pip install -q -U torch transformers bitsandbytes accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 2. Configuração para rodar em 4-bits (Leve, para o Colab não travar)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 3. Carregar o Modelo e o Tokenizer do Hugging Face
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

print("Carregando modelo... (Isso pode levar uns 2 ou 3 minutos)")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 4. Definir o seu Contexto (A "Verdade" que você coletou)
contexto_projeto = """
DADOS DO PROJETO MoneyPrinterTurbo:

1. BRANCHES E FLUXO OBSERVADO:
   - Branches principais identificadas: 'main' e 'dev'.
   - O autor trabalha diretamente na branch 'dev'.
   - Pull Requests são abertos da 'dev' para a 'main' quando deseja subir alterações estáveis.
   - Contribuidores externos (forks) criam branches de funcionalidade (ex: 'feat/content-generation'), mas o autor principal foca em dev->main.

2. HISTÓRICO DE COMMITS/PRs RECENTES:
   - PR #692: "Merge pull request #692 from harry0703/dev" (Integração da dev na main).
   - Commit: "feat: update bug report and feature request templates".
   - PR de terceiro: "fransievn wants to merge 1 commit into harry0703:main from fransievn:feat/content-generation".

3. TAGS/RELEASES (Estratégia Temporal):
   - v1.2.6 (Latest) - Traz melhorias como "SiliconFlow TTS services".
   - Releases anteriores não seguem data fixa (ex: não é semanal/mensal).
   - Releases parecem ser criadas quando há features novas ou correções críticas acumuladas na main.
"""

# 5. Criar o Prompt no formato que o Mistral entende
# O formato é: <s>[INST] Instrução + Contexto [/INST]
prompt_texto = f"""<s>[INST]
Atue como um Especialista em DevOps. Analise os dados brutos abaixo do repositório 'MoneyPrinterTurbo'.

{contexto_projeto}

Com base APENAS nesses dados, responda:
1. Qual é o Modelo de Fluxo de Trabalho (Branching Model) utilizado? (Ex: Gitflow, GitHub Flow, Trunk Based, Customizado?)
2. Qual é a Estratégia de Releases utilizada? (Ex: Release Train, Feature-based, Rapid Release?)

Justifique sua resposta citando as evidências dos dados.
[/INST]"""

# 6. Gerar a resposta
inputs = tokenizer(prompt_texto, return_tensors="pt").to("cuda")

print("Gerando análise...")
outputs = model.generate(
    **inputs,
    max_new_tokens=1000, # Tamanho máximo da resposta
    do_sample=True,
    temperature=0.1,     # Temperatura baixa para ser mais "técnico" e menos criativo
    top_p=0.95
)

# 7. Decodificar e imprimir o resultado limpo
resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Remove a parte do prompt para mostrar só a resposta nova
print("-" * 30)
print(resposta.split("[/INST]")[1])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Gerando análise...
------------------------------
 1. Based on the provided data, it appears that the 'MoneyPrinterTurbo' repository is using a GitHub Flow branching model. This is inferred from the description of the workflow where the author works directly on the 'dev' branch and merges it into the 'main' branch via pull requests. This model allows for continuous integration and delivery, while also enabling the team to release stable versions to production when they are ready.

2. The release strategy for 'MoneyPrinterTurbo' seems to be feature-based. This is inferred from the fact that releases do not follow a fixed schedule and are created when there are new features or critical bug fixes accumulated in the 'main' branch. This strategy allows the team to release new versions whenever they have significant changes to deliver to their users.

Evidence:
- GitHub Flow branching model: The author works on the 'dev' branch and merges it into 'main' via pull requests.
- Feature-based rel